# Descarga de datos

Este cuaderno procesa los datos descargados de GBIF, que utiliza la base de datos de INaturalist. En concreto, procesa el dataset que se puede descargar del siguiente enlace:

https://static.inaturalist.org/observations/gbif-observations-dwca.zip

El zip descargado ocupa más de 17 GB, y, una vez extraído, hay dos archivos csv de más de 68 y 46 GB, respectivamente, por lo que no se pueden cargar en memoria ni abrir directamente.

También se puede descargar y extraer ejecutando las siguientes celdas:

In [ ]:
import requests
import zipfile
import io

In [ ]:
# URL of the ZIP file
url = "https://static.inaturalist.org/observations/gbif-observations-dwca.zip"

# Download the file
response = requests.get(url)
response.raise_for_status() # Raise an error if download fails

# Unzip it
with zipfile.ZipFile(io.BytesIO(response.content)) as z:
    z.extractall("Datos iNaturalist")

# Filtrado de datos

In [ ]:
import os
import pandas as pd
import shutil

In [ ]:
chunk = pd.read_csv('Datos iNaturalist/media.csv', chunksize=1000)
for c in chunk:
    df = c
    break
df.to_excel('Datos iNaturalist/media_sample.xlsx', index=False)
chunk = pd.read_csv('Datos iNaturalist/observations.csv', chunksize=1000)
for c in chunk:
    df = c
    break
df.to_excel('Datos iNaturalist/observations_sample.xlsx', index=False)

Observando las hojas de cálculo resultantes, vemos que el ID es compartido entre ambas tablas.

La tabla media contiene información sobre la imagen y proporciona un link directo, mientras que la tabla observations contiene información sobre el contexto en el que se tomó la foto, así como sus licencias:

In [ ]:
# Eliminar los archivos anteriores:
os.remove('Datos iNaturalist/media_sample.xlsx')
os.remove('Datos iNaturalist/observations_sample.xlsx')

In [ ]:
chunk = pd.read_csv('Datos iNaturalist/observations.csv', chunksize=1000)
country_codes = set()
licenses = {} # {license: count}
for c in chunk:
    country_codes.update(c['countryCode'])
    for license in c['license']:
        if license in licenses:
            licenses[license] += 1
        else:
            licenses[license] = 1
    print(len(country_codes), end='\r')
    if len(country_codes) >= 240:
        break

In [26]:
print(country_codes)

{'ZA', 'AG', 'VU', 'DE', 'AF', 'CC', 'NZ', 'BS', 'MP', 'JE', 'QA', 'SY', 'SE', 'VN', 'DZ', 'MX', 'MM', 'JO', 'TG', 'CG', 'NL', 'KR', 'PH', 'IN', 'FI', 'DO', 'SC', 'LR', 'RU', 'CU', 'GD', 'AE', 'GY', 'KG', 'GT', 'BI', 'BB', 'FK', 'FR', 'GP', 'UM', 'AL', 'SL', 'MD', 'LV', 'ZW', 'ST', 'TD', 'IS', 'TT', 'UZ', 'CV', 'SN', 'ID', 'UA', 'PK', 'PN', 'AX', 'TC', 'US', 'HR', 'AO', 'IQ', 'TL', 'MV', 'SH', 'AU', 'KH', 'MY', 'LU', 'GE', 'MC', 'YE', 'SG', 'SB', 'BN', 'KN', 'SI', 'CN', 'GM', 'NE', 'GQ', 'MS', 'LY', 'VC', 'MR', 'BM', 'MO', 'ER', 'CR', 'GU', 'MZ', 'NO', 'PA', 'TH', 'PY', 'GG', 'JM', 'LA', 'SR', 'HN', 'MA', 'BH', 'MF', 'PM', 'PR', 'GA', 'HT', 'KI', 'DM', 'CZ', 'BQ', 'ME', 'ZM', 'SV', 'AZ', 'SO', 'BE', 'EG', 'GW', nan, 'SJ', 'BD', 'HK', 'BJ', 'CM', 'TV', 'RO', 'VE', 'MK', 'CF', 'RS', 'HU', 'KZ', 'TZ', 'SZ', 'TN', 'CO', 'IL', 'AR', 'AQ', 'BY', 'BW', 'MG', 'TO', 'CA', 'BL', 'SA', 'AD', 'NI', 'GN', 'VG', 'PT', 'RE', 'MW', 'LS', 'AM', 'SS', 'GH', 'LI', 'PG', 'NF', 'LT', 'SX', 'BA', 'MU', 'TJ'

In [ ]:
print(licenses)

{'http://creativecommons.org/publicdomain/zero/1.0/': 59970, 'http://creativecommons.org/licenses/by/4.0/': 155099, 'http://creativecommons.org/licenses/by-nc/4.0/': 755858, 'http://creativecommons.org/licenses/by-nd/4.0/': 9, 'http://creativecommons.org/licenses/by-sa/4.0/': 48, nan: 13, 'http://creativecommons.org/licenses/by-nc-sa/4.0/': 3}


Podemos observar que las licencias hacen referencia a las de Creative Commons, que podríamos clasificar en:

- http://creativecommons.org/publicdomain/zero/1.0/
- http://creativecommons.org/licenses/by/4.0/
- http://creativecommons.org/licenses/by-nc/4.0/
- http://creativecommons.org/licenses/by-nd/4.0/
- http://creativecommons.org/licenses/by-sa/4.0/
- http://creativecommons.org/licenses/by-nc-sa/4.0/
- http://creativecommons.org/licenses/by-nc-nd/4.0/

Filtramos por código de país ES y quitamos las observaciones con licencias no comercializables. La siguiente celda tarda unos 45 minutos en ejecutarse.

In [ ]:
os.makedirs('Datos iNaturalist/observations_chunked', exist_ok=True)

# Guardar los encabezados en un archivo separado
pd.read_csv('Datos iNaturalist/observations.csv', chunksize=1).get_chunk(0).to_csv('Datos iNaturalist/observations_chunked/headers.csv', index=False)
chunks = pd.read_csv('Datos iNaturalist/observations.csv', chunksize=10000) # El argumento chunksize hace que se puedan leer archivos grandes en trozos

for i, c in enumerate(chunks):
    if (i+1) % 100 == 0:
        print(f'Processing chunk {i+1}...', end='\r')
    df = c[(c['countryCode'] == 'ES') &
           (c['license'] != 'http://creativecommons.org/licenses/by-nc/4.0/') &
           (c['license'] != 'http://creativecommons.org/licenses/by-nc-sa/4.0/') &
           (c['license'] != 'http://creativecommons.org/licenses/by-nc-nd/4.0/') &
           (c['license'] != 'http://creativecommons.org/licenses/by-nc-nd/4.0/')]
    if len(df) > 0:
        df.to_csv(f'Datos iNaturalist/observations_chunked/{i}.csv', index=False, header=False)
print(f'All 10868 chunks processed.')

All 10868 chunks processed.


Vamos a combinar todas las observaciones en un solo archivo:

In [ ]:
with open('Datos iNaturalist/observations_selection.csv','wb') as fout:
    # Archivo con encabezados
    fout.write(open('Datos iNaturalist/observations_chunked/headers.csv','rb').read())
    # Archivos con datos
    for filename in os.listdir('Datos iNaturalist/observations_chunked'):
        if filename == 'headers.csv':
            continue
        with open(f'Datos iNaturalist/observations_chunked/{filename}', 'rb') as fin:
            for line in fin:
                fout.write(line)
shutil.rmtree("Datos iNaturalist/observations_chunked")

Con esto, nos sale un archivo de más de 260.000 observaciones:

In [ ]:
ids = pd.read_csv('Datos iNaturalist/observations_selection.csv')['id']
len(ids)

261793

Ahora, obtenemos la información del archivo media.csv correspondiente a las observaciones que hemos seleccionado. Parece que muchas imágenes tienen una licencia que impide la comercialización incluso aunque la observación esté marcada como que sí, por lo que debemos tener eso en cuenta a la hora de filtrar.

Con todo ello, la siguiente celda tarda unos 35 minutos en ejecutarse:

In [ ]:
os.makedirs('Datos iNaturalist/media_chunked', exist_ok=True)

# Guardar los encabezados en un archivo separado
pd.read_csv('Datos iNaturalist/media.csv', chunksize=1).get_chunk(0).to_csv('Datos iNaturalist/media_chunked/headers.csv', index=False)
chunks = pd.read_csv('Datos iNaturalist/media.csv', chunksize=10000) # El argumento chunksize hace que se puedan leer archivos grandes en trozos

for i, c in enumerate(chunks):
    if (i+1) % 100 == 0:
        print(f'Processing chunk {i+1}...', end='\r')
    df = c[c['id'].isin(ids) &
           (c['license'] != 'http://creativecommons.org/licenses/by-nc/4.0/') &
           (c['license'] != 'http://creativecommons.org/licenses/by-nc-sa/4.0/') &
           (c['license'] != 'http://creativecommons.org/licenses/by-nc-nd/4.0/') &
           (c['license'] != 'http://creativecommons.org/licenses/by-nc-nd/4.0/')]
    if len(df) > 0:
        df.to_csv(f'Datos iNaturalist/media_chunked/{i}.csv', index=False, header=False)
print(f'All {i+1} chunks processed.')

All 18108 chunks processed.


Como vemos, hay más fragmentos que en el archivo anterior, lo que se debe a que cada observación puede tener más de una imagen asociada. Por tanto, es lógico que haya más filas en el archivo de imágenes que en el de observaciones.

Combinamos de nuevo los resultados en un solo archivo:

In [ ]:
with open('Datos iNaturalist/media_selection.csv','wb') as fout:
    # Archivo con encabezados
    fout.write(open('Datos iNaturalist/media_chunked/headers.csv','rb').read())
    # Archivos con datos
    for filename in os.listdir('Datos iNaturalist/media_chunked'):
        if filename == 'headers.csv':
            continue
        with open(f'Datos iNaturalist/media_chunked/{filename}', 'rb') as fin:
            for line in fin:
                fout.write(line)
shutil.rmtree("Datos iNaturalist/media_chunked")

Con esto, podemos ver el número de imágenes disponibles:

In [ ]:
len(pd.read_csv('Datos iNaturalist/media_selection.csv'))

365714

En total, habría 473.583 imágenes si no hubiéramos realizado el filtrado de licencias, pero se quedan en 365.714 si sí lo tenemos en cuenta.

Por último, debemos eliminar aquellas observaciones para las que no queda ninguna imagen porque todas las asociadas a esa observación tenían una licencia no comercializable.

In [ ]:
observations_df = pd.read_csv('Datos iNaturalist/observations_selection.csv')
media_df = pd.read_csv('Datos iNaturalist/media_selection.csv')
observations_df = observations_df[observations_df['id'].isin(media_df['id'].unique())]
observations_df.to_csv('Datos iNaturalist/observations_selection.csv', index=False)
len(observations_df)

181485

Observamos que hemos reducido en gran medida el número de observaciones que podemos usar, de 261.793 a 181.485.

### Eliminar columnas innecesarias

La tabla de observaciones contiene muchas columnas que solo toman un único valor, así que podemos eliminarlas ya que no aportan nada a los datos.

In [ ]:
df = pd.read_csv('Datos iNaturalist/observations_selection.csv')
for col in df.columns:
    if col == 'countryCode':
        continue
    counts = df[col].value_counts()
    if len(counts) == 1 and len(df) == counts.values[0]:
        print(f'Dropping column {col} because all records have the value {counts.index[0]}')
        df.drop(col, axis=1, inplace=True)
if len(df[df['id'] == df['catalogNumber']]) == len(df):
    print('Dropping column catalogNumber because it is the same as id')
    df.drop('catalogNumber', axis=1, inplace=True)
df.to_csv('Datos iNaturalist/observations_selection.csv', index=False)

Dropping column basisOfRecord because all records have the value HumanObservation
Dropping column institutionCode because all records have the value iNaturalist
Dropping column collectionCode because all records have the value Observations
Dropping column datasetName because all records have the value iNaturalist research-grade observations
Dropping column captive because all records have the value wild
Dropping column geodeticDatum because all records have the value EPSG:4326
Dropping column catalogNumber because it is the same as id
